In [1]:
import gc
import os
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from tqdm import tqdm
import catboost as cb
from datetime import datetime
from sklearn.metrics import f1_score

from utils import seed_everything
from metrics_f1 import calc_f1_score

DATA_ROOT = r"./"
WEIGHTS_ROOT = "weights"
os.makedirs(WEIGHTS_ROOT, exist_ok=True)

SEED = 28

seed_everything(SEED)
date = str(datetime.now()).split('.')[0].replace(':','_').replace(' ','__').replace('-','_')

gt_path = 'prediction/target_predicton_true.csv'
pred_path = 'prediction/target_predicton.csv'

baseline_score = calc_f1_score(gt_path, pred_path)
print(f"{baseline_score = }")

baseline_score = 0.23121339736861674


In [2]:
numerical_features, categorical_features, text_features = [], [], []
months_val = [pd.to_datetime('2022-12-01')]

In [3]:
target1_df = pd.read_csv('target/y_train.csv').convert_dtypes()
target2_df = pd.read_csv('../train2/target/y_test.csv').convert_dtypes()

target_df = pd.concat([target1_df, target2_df])
target_df = target_df.sort_values(by=["wagnum", "month"])
target_df = target_df.drop_duplicates(ignore_index=True)
target_df.month = pd.to_datetime(target_df.month)

print(target_df["month"].value_counts())

del target1_df, target2_df
gc.collect()

month
2022-12-01    33977
2022-09-01    33976
2022-10-01    33976
2022-11-01    33976
2022-08-01    33975
2023-01-01    33973
2023-02-01    33708
Name: count, dtype: int64


0

In [4]:
wag_prob1_df = pd.read_parquet('wagons_probeg_ownersip.parquet').convert_dtypes()
wag_prob2_df = pd.read_parquet('wagons_probeg_ownersip.parquet').convert_dtypes()

wag_prob_df = pd.concat([wag_prob1_df, wag_prob2_df])
wag_prob_df = wag_prob_df.sort_values(by=["wagnum", "repdate"])
wag_prob_df = wag_prob_df.drop_duplicates(ignore_index=True)
wag_prob_df = wag_prob_df.drop(columns=["month"])

del wag_prob1_df, wag_prob2_df
gc.collect()

20

In [5]:
dislok1_df = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
dislok2_df = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()

dislok_df = pd.concat([dislok1_df, dislok2_df])
dislok_df = dislok_df.sort_values(by=["wagnum", "plan_date"])
dislok_df = dislok_df.drop_duplicates(ignore_index=True)

del dislok1_df, dislok2_df
gc.collect()

0

In [6]:
wag_param1_df = pd.read_parquet('wag_params.parquet').convert_dtypes()
wag_param2_df = pd.read_parquet('wag_params.parquet').convert_dtypes()

wag_param_df = pd.concat([wag_param1_df, wag_param2_df])
wag_param_df = wag_param_df.sort_values(by=["wagnum"])
wag_param_df = wag_param_df.drop_duplicates(ignore_index=True)

del wag_param1_df, wag_param2_df
gc.collect()

0

In [7]:
pr_rem1_df = pd.read_parquet('pr_rems.parquet').convert_dtypes()
pr_rem2_df = pd.read_parquet('pr_rems.parquet').convert_dtypes()

pr_rem_df = pd.concat([pr_rem1_df, pr_rem2_df])
pr_rem_df = pr_rem_df.sort_values(by=["wagnum", "rem_month"])
pr_rem_df = pr_rem_df.drop_duplicates(ignore_index=True)

del pr_rem1_df, pr_rem2_df
gc.collect()

0

In [8]:
tr_rem1_df = pd.read_parquet('tr_rems.parquet').convert_dtypes()
tr_rem2_df = pd.read_parquet('tr_rems.parquet').convert_dtypes()

tr_rem_df = pd.concat([tr_rem1_df, tr_rem2_df])
tr_rem_df = tr_rem_df.sort_values(by=["wagnum", "rem_month"])
tr_rem_df = tr_rem_df.drop_duplicates(ignore_index=True)

del tr_rem1_df, tr_rem2_df
gc.collect()

0

In [9]:
kti_izm1_df = pd.read_parquet('kti_izm.parquet').convert_dtypes()
kti_izm2_df = pd.read_parquet('kti_izm.parquet').convert_dtypes()

kti_izm_df = pd.concat([kti_izm1_df, kti_izm2_df])
kti_izm_df = kti_izm_df.sort_values(by=["wagnum", "operation_date_dttm"])
kti_izm_df = kti_izm_df.drop_duplicates(ignore_index=True)
kti_izm_df["operation_date_dttm"] = kti_izm_df["operation_date_dttm"].apply(lambda x: datetime.fromtimestamp(int(str(x)[:-9])).replace(hour=0, minute=0, second=0, microsecond=0))

del kti_izm1_df, kti_izm2_df
gc.collect()

0

In [10]:
freight_info1_df = pd.read_parquet('freight_info.parquet').convert_dtypes()
freight_info2_df = pd.read_parquet('freight_info.parquet').convert_dtypes()

freight_info_df = pd.concat([freight_info1_df, freight_info2_df])
freight_info_df = freight_info_df.sort_values(by=["fr_id"])
freight_info_df = freight_info_df.drop_duplicates(ignore_index=True)

del freight_info1_df, freight_info2_df
gc.collect()

0

In [11]:
stations1_df = pd.read_parquet('stations.parquet').convert_dtypes()
stations2_df = pd.read_parquet('stations.parquet').convert_dtypes()

stations_df = pd.concat([stations1_df, stations2_df])
stations_df = stations_df.sort_values(by=["st_id"])
stations_df = stations_df.drop_duplicates(ignore_index=True)

del stations1_df, stations2_df
gc.collect()

0

In [12]:
df = pd.merge(target_df, wag_prob_df, how='left', on=["wagnum"])
df = df.drop(df[df["repdate"] > df["month"]].index)
df = df.sort_values(by=["wagnum", "month", "repdate"])
df = df.groupby(["wagnum", "month"]).last().reset_index()
df = df.fillna(-1)

df["ost_prob"] = df["ost_prob"].astype(int)
df["manage_type"] = df["manage_type"].astype(int)
df["rod_id"] = df["rod_id"].astype(int)
df["reestr_state"] = df["reestr_state"].astype(int)
numerical_features.extend(["ost_prob", "manage_type", "rod_id", "reestr_state"])

In [13]:
def fit_cb(df, params, months_val, target_name):

    x_train = df[~df["month"].isin(months_val)][numerical_features + categorical_features + text_features]
    x_val = df[df["month"].isin(months_val)][numerical_features + categorical_features + text_features]

    y_train = df[~df["month"].isin(months_val)][target_name]
    y_val = df[df["month"].isin(months_val)][target_name]
    print(f"Train class imbalance: {y_train.value_counts()}")
    print(f"Val class imbalance: {y_val.value_counts()}")

    train_pool = cb.Pool(
        data = x_train,
        label = y_train,
        cat_features = categorical_features,
        text_features = text_features
    )

    eval_pool = cb.Pool(
        data = x_val,
        label = y_val,
        cat_features = categorical_features,
        text_features = text_features
    )

    model = cb.CatBoostClassifier(**params)

    model.fit(
        train_pool,
        eval_set=eval_pool,
        verbose=True
    )
    print("best results (train on train):")
    print(model.get_best_score()["learn"])
    print("best results (on validation set):")
    print(model.get_best_score()["validation"])
    print(model.get_feature_importance(data=train_pool, prettified=True))

    return model

In [16]:
cb_params  = {
    'iterations': 2000,
    'loss_function': 'CrossEntropy',
    'custom_metric': ['AUC', 'Accuracy', 'F1'],
    'verbose': False,
    'random_seed': SEED,
    "task_type": "CPU",
    "has_time": True,
    "metric_period": 500,
    "save_snapshot": False,
    "use_best_model": True,
}

cb_month_model = fit_cb(df, cb_params, months_val=months_val, target_name="target_month")
cb_10days_model = fit_cb(df, cb_params, months_val=months_val, target_name="target_day")

Train class imbalance: target_month
0    193178
1     10405
Name: count, dtype: Int64
Val class imbalance: target_month
0    32393
1     1584
Name: count, dtype: Int64
0:	learn: 0.6215861	test: 0.6186859	best: 0.6186859 (0)	total: 72.4ms	remaining: 2m 24s
500:	learn: 0.1024499	test: 0.0927711	best: 0.0927711 (500)	total: 5.96s	remaining: 17.8s
1000:	learn: 0.1021366	test: 0.0928816	best: 0.0927711 (500)	total: 11.8s	remaining: 11.8s
1500:	learn: 0.1019929	test: 0.0929431	best: 0.0927711 (500)	total: 17.8s	remaining: 5.9s
1999:	learn: 0.1019221	test: 0.0929870	best: 0.0927711 (500)	total: 23.3s	remaining: 0us

bestTest = 0.09277114193
bestIteration = 500

Shrink model to first 501 iterations.
best results (train on train):
{'Accuracy': 0.9591370595776662, 'F1': 0.4778816590324402, 'CrossEntropy': 0.10192207208890892}
best results (on validation set):
{'Accuracy': 0.962886658622009, 'F1': 0.49194847020933974, 'CrossEntropy': 0.09277114193049793, 'AUC': 0.9609464625884069}
     Feature Id

In [17]:
x_val = df[df["month"].isin(months_val)][numerical_features + categorical_features + text_features]

pred_month = cb_month_model.predict(x_val)
pred_10days = cb_10days_model.predict(x_val)

In [18]:
val_df = pd.DataFrame({"wagnum": df[df["month"].isin(months_val)]["wagnum"]})
val_df["target_month"] = pred_month
val_df["target_day"] = pred_10days

In [19]:
def my_calc_f1_score(gt_path, pred_df):

    pred_labels = pred_df.sort_values(by=["wagnum"])

    true_labels = pd.read_csv(gt_path)
    true_labels = true_labels.sort_values(by=["wagnum"])

    # Таргет для месячного прогноза
    true_labels_month = true_labels['target_month'].values
    pred_labels_month = pred_labels['target_month'].values

    # Таргет для 10 дневного прогноза
    true_labels_day = true_labels['target_day'].values
    pred_labels_day = pred_labels['target_day'].values

    # Посчитаем метрику для месяца и 10 дней
    score_month = f1_score(true_labels_month, pred_labels_month)
    score_day = f1_score(true_labels_day, pred_labels_day)

    # Посчитаем метрику с весом для двух таргетов
    score = 0.5 * score_month + 0.5 * score_day
    return score

cv_score = my_calc_f1_score(gt_path, val_df)
print(f"{cv_score = }")

cv_score = 0.32735441156600464


In [21]:
test_df = pd.read_csv(DATA_ROOT + '../train/target/y_predict.csv').convert_dtypes()
test_df_copy = test_df.copy()

test_df = pd.merge(test_df, wag_prob_df, how='left', on=["wagnum"])
test_df = test_df.drop(test_df[test_df["repdate"] > test_df["month"]].index)
test_df = test_df.groupby(["wagnum", "month"]).last().reset_index()
test_df = test_df.fillna(-1)
test_df = test_df_copy.merge(test_df)

test_df["ost_prob"] = test_df["ost_prob"].astype(int)
test_df["manage_type"] = test_df["manage_type"].astype(int)
test_df["rod_id"] = test_df["rod_id"].astype(int)
test_df["reestr_state"] = test_df["reestr_state"].astype(int)

In [22]:
x_test = test_df[numerical_features + categorical_features + text_features]

pred_month = cb_month_model.predict(x_test)
pred_10days = cb_10days_model.predict(x_test)

In [23]:
submit_df = test_df[["wagnum", "month"]]
submit_df["target_month"] = pred_month
submit_df["target_day"] = pred_10days

print(submit_df["target_month"].value_counts())
print(submit_df["target_day"].value_counts())

submit_df.to_csv("zalupa.csv", index=False)

target_month
0    32848
1      859
Name: count, dtype: int64
target_day
0    33600
1      107
Name: count, dtype: int64


/var/folders/wp/ll6ymdkx1gg8l2vyp57k2r840000gn/T/ipykernel_18616/2387483953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_df["target_month"] = pred_month
/var/folders/wp/ll6ymdkx1gg8l2vyp57k2r840000gn/T/ipykernel_18616/2387483953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_df["target_day"] = pred_10days
